In [1]:
!pip install "/kaggle/input/autocorrect/autocorrect-2.6.1.tar"
!pip install /kaggle/input/textstat-pypi/Pyphen-0.9.3-py2.py3-none-any.whl
!pip install /kaggle/input/textstat-pypi/textstat-0.7.0-py3-none-any.whl

Processing /kaggle/input/autocorrect/autocorrect-2.6.1.tar
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622383 sha256=3f3e31a360e89527497ea7153cf905935de76faa041c4b756a9a9cb8208dec7c
  Stored in directory: /root/.cache/pip/wheels/db/69/42/0fb0421d2fe70d195a04665edc760cfe5fd341d7bb8d8e0aaa
Successfully built autocorrect
Processing /kaggle/input/textstat-pypi/Pyphen-0.9.3-py2.py3-none-any.whl
Processing /kaggle/input/textstat-pypi/textstat-0.7.0-py3-none-any.whl


- 3-12학년 학생들이 작성한 요약문의 퀄리티 평가
- 평가 요소: main idea, details of a source text, clarity, precision, fluency
- 요약문 평가는 학생 요약문과 요약문의 source text 둘다 고려해야해서 복잡함

Textstat is an easy to use library to calculate statistics from text. It helps determine readability, complexity, and grade level.

In [6]:
import os

import textstat
import numpy as np
import pandas as pd

In [7]:
def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

prompts_train = 4개의 traning set prompmts. 각 프롬프트는 학생들에게 주어진 완전한 요약 과제들로 구성되어 있다.

* prompt_id - 요약파일과 연결되는 프롬프트 아이디
* prompt_questions - 학생들이 답해야하는 질문
* prompt_title - 프롬프트의 짧은 버전 제목
* prompt_text - the full prompt text

In [8]:
prompts_train = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv")
prompts_test = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv")

summaries_train = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv")
summaries_test = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv")

sample_submission = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv")

prompts_train

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [9]:
# dataframe 병합, prompt id 기준으로 각 아이디에 맞는 prompts_train, test 내용을 왼쪽에 붙임
train = summaries_train.merge(prompts_train, how="left", on="prompt_id")
test = summaries_test.merge(prompts_test, how="left", on="prompt_id")

train

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...


In [10]:
test

,student_id,prompt_id,text,prompt_question,prompt_title,prompt_text
0,000000ffffff,abc123,Example text 1,Summarize...,Example Title 1,Heading\nText...
1,111111eeeeee,def789,Example text 2,Summarize...,Example Title 2,Heading\nText...
2,222222cccccc,abc123,Example text 3,Summarize...,Example Title 1,Heading\nText...
3,333333dddddd,def789,Example text 4,Summarize...,Example Title 2,Heading\nText...


Let's create some features using text statistics and use classic ML algorithm

ml알고리즘 쓴다는데 왜 허깅페이스에서 모델 가져오지??

In [11]:
from transformers import AutoTokenizer, AutoModel,AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

AutoTokenizer, Model
* 모든 클래스에 걸쳐 Auto 클래스가 존재
* 모델에 관한 정보를 처음부터 명시하지 않아도 되어 조금 유용하게 사용 가능
* 모델의 상세정보를 확인할 필요 없이 Model ID (ex:bert-base-cased) 만으로도 손쉽게 모델 구성이 가능
* 정확한 용도에 맞게 사용하려면 모델별 상세 안내 페이지를 참고해서 최적의 모델 선택

Sentence Bert는 Bert에 Sentence Embedding을 가능하도록 파인튜팅한 모델
 * sentence embedding이란, 문장을 벡터공간에 배치하는 방법을 의미함. 문장을 벡터 공간 내 배치함으로서 문장비교, 클러스터링, 시각화 등 다양한 방법을 적용할 수 있음.

In [12]:
# Load model from HuggingFace Hub
tokenizer_sb = AutoTokenizer.from_pretrained('/kaggle/input/sentence-transformers/sentence-transformers/minil2-l12-v2/paraphrase-multilingual-MiniLM-L12-v2')
model_sb = AutoModel.from_pretrained('/kaggle/input/sentence-transformers/sentence-transformers/minil2-l12-v2/paraphrase-multilingual-MiniLM-L12-v2')
MEMORY_SB = {}

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

multi-qa-mpnet-base-dot-v1

It maps sentences & paragraphs to a 768 dimensional dense vector space and was designed for semantic search. It has been trained on 215M (question, answer) pairs from diverse sources.

 문장과 문단들을 768 차원의 벡터 공간에 매핑함. semantic search를 위해 고안됨. 215M의 QA 페어로 훈련됨

In [13]:
tokenizer_qa = AutoTokenizer.from_pretrained('/kaggle/input/multi-qa-mpnet-base-dot-v1/multi-qa-mpnet-base-dot-v1')
model_qa = AutoModel.from_pretrained('/kaggle/input/multi-qa-mpnet-base-dot-v1/multi-qa-mpnet-base-dot-v1')
MEMORY_QA = {}

뭐한는 놈?? bert_cohesion?

This model is a fine-tuned version of bert-base-uncased on an unknown dataset. It achieves the following results on the evaluation set:

Loss: 1.7082

Accuracy: 0.4

F1: 0.2857


버트기반 단어 응집도 측정?

응집도란: 응결성(Cohesion)은 한 텍스트나 문장에 다른 한 텍스트가 포함되어 있고, 그 포함된 텍스트에 의미가 부여되는 경우, 전체 텍스트 내에서 보이는 문법적 연결성(grammatical linking) 혹은 어휘적 연결성(lexical linking)을 말한다. 

In [12]:
tokenizer_ch = AutoTokenizer.from_pretrained('/kaggle/input/bert-cohesion/bert_cohesion')
model_ch = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/bert-cohesion/bert_cohesion', num_labels=9) # 라벨이 9개긴 함.

Reward model (RM) trained to predict which generated answer is better judged by a human, given a question.
질문이 주어졌을 때 어떤 답변이 사람이 더 잘 판단할 수 있는지 예측하도록 학습된 보상 모델

RM are useful in these domain:

QA model evaluation

serves as reward score in RLHF

detect potential toxic response via ranking

In [13]:
# num_labels의 기준?? 모델 안에 정해진 label이 1개여서 그런ㄱ가.
tokenizer_rw = AutoTokenizer.from_pretrained('/kaggle/input/reward-model-deberta-v3-large-v2-wch/reward-model-deberta-v3-large-v2-wch')
model_rw = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/reward-model-deberta-v3-large-v2-wch/reward-model-deberta-v3-large-v2-wch',
                                                              num_labels=1)

* torch.sum(a) - dim 없으면 matrix의 모든 값을 더한 값
* torch.sum(a, dim) - dim=0이면, column 기준으로 더함, dim=1이면 row 기준으로 더함

사전학습 된 BERT로부터 문장벡터를 얻는 방법
1. BERT의 [CLS] 토큰의 출력 벡터를 문장 벡터로 간주
2. BERT의 모든 단어의 출력 벡터에 대해서 평균 풀링을 수행한 벡터를 문장 벡터로 간주한다.
3. BERT의 모든 단어의 출력 벡터에 대해서 맥스 풀링을 수행한 벡터를 문장 벡터로 간주한다.

* 평균풀링을 얻은 문장 벡터 - 모든 단어의 의미를 반영
* 맥스풀링을 얻은 문장 벡터 - 중요한 단어의 의미를 반영

일반적으로 버트 기반(트랜스포머 전체인지는 잘 모름) 토크나이저 출력값은 총 3개로 나뉨.

* input_ids - token들의 id 리스트
* token_type_ids - 각 token이 어떤 문장에 속하는지를 나타내는 리스트. BERT는 한 번에 두 문장(sentence A, sentence B)을 입력으로 받을 수 있는데, bert-base-uncased tokenizer는 sentence A에 속하는 token에는 0을, sentence B에 속하는 token에는 1을 부여함
* attention_mask - attention 연산이 수행되어야 할 token과 무시해야 할 token을 구별하는 정보가 담긴 리스트. bert-base-uncased tokenizer는 attention 연산이 수행되어야 할, 일반적인 token에는 1을 부여하고, padding과 같이 attention 연산이 수행될 필요가 없는 token들에는 0을 부여한다.

In [14]:
#Mean Pooling - Take attention mask into account for correct averaging
# multilingual 모델에서 사
def mean_pooling(model_output, attention_mask):
    '''문장 내 토큰을 합한 뒤 평균'''
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings/model_output의 첫번재 요소에 all token embedding있음
    # padding 부분 찾기 =[batch_size, src_token, embed_size]
    # attention_mask의 맨 뒷부분에 1로 된 차원 추가하는데 token_embedding만큼의 크기를 추가함.
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    # torch.sum(model의 all token embeeding 개수 / 평균 내기 위한 token 개수)
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# QA 모델에서 사용
def cls_pooling(model_output):
    # [cls](문장 시작) 부분만 추출 [batch_size, embed_size]
    # .last_hidden_state는 문장 n개의 입력 토큰 각각에 해당하는 BERT의 마지막 레이어 출력 벡터들이다.
    return model_output.last_hidden_state[:,0] # Sequence of hidden-states at the output of the last layer of the model

from transformers import AutoTokenizer, AutoModel,AutoModelForSequenceClassification

import torch

import torch.nn.functional as F

이 때 마지막 노드에서 아무런 Activation Function을 거치지 않은 값을 Logit이라함.

In [15]:
 # torch.no_grad() -> gradient 계산 context 비활성화
    # Inference나 validation에서는 gradient를 계산 안하기 때문에 이 함수를 사용함으로써 
    # Pytorch의 autograd engine(gradient를 계산해주는 context)를 비활성화 시켜 gradient를 트래킹 하지 않음
    # 이를 통해 필요한 메모리가 줄어들고 연산속도도 증가
    # 정리 torch.no_grad() 함수는 autograd engine을 비활성화 시켜 필요한 메모리를 줄이고 연산속도 증가시킴
    
# 의문1 왜 기본 sbert와 qa용 두개 를 썼을까?
def get_emb_sb(sentences):
    # sentence bert 임베딩, 이미 문장이 memory에 있다면 해당 key 반환
    if sentences in MEMORY_SB:
        return MEMORY_SB[sentences]
    # Tokenize sentences
    encoded_input = tokenizer_sb([sentences], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model_sb(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings L2, 1차원으로 reduce
    # 데이터의 범위를 사용자가 원하는 범위로 제한 -> Normalize하면 학습을 더 빠르게 하며, Local Optimum에 빠지는 가능성을 줄임
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)[0].detach().cpu().numpy() # cpu로 바꿈
    MEMORY_SB[sentences] = sentence_embeddings
    
    return sentence_embeddings

def get_emb_qa(sentences):
    if sentences in MEMORY_QA:
        return MEMORY_QA[sentences]
    # Tokenize sentences
    # padding - 길이가 다양한 문장들을 병렬 연산을 위해 임의로 동일하게 맞춰줌 -> max_length에 맞
    # truncation - 지정된 최대 문장 길이보다 길 경우 문장 잘림 허용 
    encoded_input = tokenizer_qa([sentences], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model_qa(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)[0].detach().cpu().numpy()
    MEMORY_QA[sentences] = sentence_embeddings
    
    return sentence_embeddings

def get_ch_label(sentences):
    encoded_input = tokenizer_ch([sentences], padding=True, truncation=True, return_tensors='pt')
    
    with torch.no_grad():
        model_output = model_ch(**encoded_input)["logits"]
    
#     print(model_output)
    label = torch.argmax(model_output, dim=1)[0].detach().cpu().numpy()
    
    return label

def get_rw(question, answer):
    # 질문과 작성한 답변을 받아서 답변이 질문에 맞게 작성되었는지 평가
    encoded_input = tokenizer_rw(question, answer, padding=True, truncation=True, return_tensors='pt')
    
    with torch.no_grad():
        model_output = model_rw(**encoded_input)["logits"]
    
    return torch.sigmoid(model_output)[0].cpu().detach().numpy()

s = f"{train.iloc[2].prompt_question} [SEP] {train.iloc[2].text}"
print(s)
get_rw(train.iloc[2].prompt_question, train.iloc[2].text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In complete sentences, summarize the structure of the ancient Egyptian system of government. How were different social classes involved in this government? Cite evidence from the text. [SEP] In Egypt, there were many occupations and social classes involved in day-to-day living. In many instances if you were at the bottom of the social ladder you could climb up, you didn't have to stay a peasant you could work to bring your status up. Everyone worshipped the gods Ra, Osiris, and Isis, but also they would worship their pharaohs like gods as well. Under the pharaohs were the priests, they had the responsibility to entertain or please the said god. The Chain of Command was placed to keep everyone in check, not one person could handle all the civilians and treasures without any aid. Like the tax collector, called a vizier like stated they were in charge of collecting the peoples' tax. They were also one of the rare instances who were able to read and write, that's how they were granted "viz

array([0.5897883], dtype=float32)

In [16]:
import nltk
from nltk.tokenize import word_tokenize


STOPWORDS = pd.read_csv("/kaggle/input/nltk-english-stopwords/nltk_eng_stopwords.csv")["list_of_stopwords"].tolist()
def get_stopwords_rel(text):
    text_words = word_tokenize(text)
    num_stopwords = sum([word in STOPWORDS for word in text_words])
    
    return num_stopwords/len(text_words)

In [17]:
[len(e.split()) for e in train['text'][0].split('.')]

[16, 10, 16, 19]

In [18]:
np.sum(([w.isupper() for w in train['text'][0].split()]))/len(train['text'][0].split())

0.0

https://pypi.org/project/textstat/
https://originality.ai/blog/what-is-the-rix-readability-formula

In [19]:
from tqdm.notebook import tqdm
tqdm.pandas()


def get_stat_features(df, text_col="text"):
    df["num_unique_words"] = df[text_col].apply(lambda x: len(set(x.split()))) # 띄어쓰기 기준 중복 단어 x
    df["num_words"] = df[text_col].apply(lambda x: len(x.split())) # 띄어쓰기 기준 전체 단어 개수
    df["num_sentences"] = df[text_col].apply(lambda x: len(x.split('.'))) # . 기준 문장 개수
    df["isupper"] = df[text_col].apply(lambda x: x[0].isupper()) # 맨 앞 요소가 대문자인 단
    df["mean_num_words"] = df[text_col].apply(lambda x: np.mean([len(e.split()) for e in x.split('.')])) # 문장 구성 단어 개수 평균
    df["mean_num_unique_words"] = df[text_col].apply(lambda x: np.mean([len(set(e.split())) for e in x.split('.')]))# 문장 구성 단어 개수 평균(중복x)
    df["num_slash"] = df[text_col].apply(lambda x: x.count("\n")) # slash?
    df["paragraph_count"] = train[text_col].apply(lambda x: x.count("\n\n")) # 문단 개수
    df["upper_count"] = df[text_col].apply(lambda x: np.sum([w.isupper() for w in x.split()])/len(x.split())) # 문장에서 대문자 단어 개수
    df["syntax_count"] = df[text_col].apply(lambda x: x.count(",") + x.count("-") + x.count(";") + x.count(":"))
    
#     df["num_unique_words_prompt_text"] = df["prompt_text"].apply(lambda x: len(set(x.split()))) - df["num_unique_words"]
#     df["num_words_prompt_text"] = df["prompt_text"].apply(lambda x: len(x.split())) - df["num_words"]
#     df["num_sentences_prompt_text"] = df["prompt_text"].apply(lambda x: len(x.split('.'))) - df["num_sentences"]
    
    df["stopwords_rel"] = df[text_col].progress_apply(lambda x: get_stopwords_rel(x))
    df['diff_emb_sb'] = df.progress_apply(lambda x: np.sum(get_emb_sb(x["text"])*get_emb_sb(x["prompt_text"])), axis=1) # text의 임베딩*prompt_text의 임베딩을 열 기준으로 더함
    df['diff_emb_qa'] = df.progress_apply(lambda x: np.sum(get_emb_qa(x["text"])*get_emb_qa(x["prompt_question"])), axis=1)
    df['ch'] = df.progress_apply(lambda x: get_ch_label(x["text"]), axis=1)
    df['rw'] = df.progress_apply(lambda x: get_rw(x["prompt_question"], x["text"]), axis=1)

    # Returns the ARI (Automated Readability Index) which outputs a number that approximates the grade level needed to comprehend the text.
    # ARI가 6.5면 6~7학년 사이의 레벨
    df['automated_readability_index'] = df[text_col].progress_apply(lambda x: textstat.automated_readability_index(x))
    # Returns the grade level of the text using the Coleman-Liau Formula. 
    # 9.3이면 9학년 수준정도면 문서를 읽을 수 있다는 뜻
    df['coleman_liau_index'] = df[text_col].progress_apply(lambda x: textstat.coleman_liau_index(x))
    # Returns the SMOG index of the given text. 
    # This is a grade formula in that a score of 9.3 means that a ninth grader would be able to read the document.
    # 30 문장 이하면 평가 어려움. 이유는 30문장 기준으로 샘플링 되어있기 때문. textstat requires at least 3 sentences for a result.
    df['smog_index'] = df[text_col].progress_apply(lambda x: textstat.smog_index(x))
    # Different from other tests, since it uses a lookup table of the most commonly used 3000 English words.
    # Thus it returns the grade level using the New Dale-Chall Formula.
    df['dale_chall_readability_score'] = df[text_col].progress_apply(lambda x: textstat.dale_chall_readability_score(x))
    # Returns the grade level using the Linsear Write Formula. This is a grade formula in that a score of 9.3 means that a ninth grader would be able to read the document.
    df['linsear_write_formula'] = df[text_col].progress_apply(lambda x: textstat.linsear_write_formula(x))
    # Returns the FOG index of the given text. This is a grade formula in that a score of 9.3 means that a ninth grader would be able to read the document.
    df['gunning_fog'] = df[text_col].progress_apply(lambda x: textstat.gunning_fog(x))
    # Based upon all the above tests, returns the estimated school grade level required to understand the text.
    # Optional float_output allows the score to be returned as a float. Defaults to False.
    df['text_standard_float'] = df[text_col].progress_apply(lambda x: textstat.text_standard(x, float_output=True))
    # Returns grade level of english text.
    # Intended for text written for children up to grade four.
    df['spache_readability'] = df[text_col].progress_apply(lambda x: textstat.spache_readability(x))
    df['rix'] = df[text_col].progress_apply(lambda x: textstat.rix(x)) # a readability formula that’s based on letter counting
    df['lix'] = df[text_col].progress_apply(lambda x: textstat.lix(x)) # Readability test for both English- and non-English-language texts

    return df
    

In [22]:
train

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text,num_unique_words,num_words,...,automated_readability_index,coleman_liau_index,smog_index,dale_chall_readability_score,linsear_write_formula,gunning_fog,text_standard_float,spache_readability,rix,lix
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,51,61,...,8.3,9.04,10.7,7.24,8.375000,9.40,9.0,4.540070,3.25,36.61
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",38,52,...,9.0,4.30,0.0,5.84,13.000000,10.40,6.0,5.001154,1.50,31.77
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,149,235,...,11.2,9.92,9.3,7.10,11.200000,9.34,9.0,4.948329,4.50,42.48
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,19,25,...,16.1,12.48,0.0,7.40,16.500000,13.20,12.0,5.740000,8.00,57.00
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,138,203,...,10.7,10.73,10.7,7.12,6.625000,10.11,11.0,4.664644,4.42,43.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",49,66,...,32.3,8.32,0.0,8.82,18.500000,28.82,9.0,11.187424,9.00,79.64
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,29,42,...,9.4,10.43,3.1,7.71,6.000000,5.60,6.0,4.655857,3.67,40.19
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,42,48,...,23.5,9.42,0.0,8.66,26.000000,19.65,9.0,8.929830,8.00,63.67
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,42,59,...,7.2,5.51,6.4,5.42,10.166667,8.56,6.0,4.049288,2.33,31.56


In [21]:
train = get_stat_features(train)
test = get_stat_features(test)

  0%|          | 0/7165 [00:00<?, ?it/s]

  0%|          | 0/7165 [00:00<?, ?it/s]

  0%|          | 0/7165 [00:00<?, ?it/s]

  0%|          | 0/7165 [00:00<?, ?it/s]

  0%|          | 0/7165 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Let's see corr matrix:

In [24]:
NO_FEATURES = ["student_id", "prompt_id", "prompt_question", "prompt_title", "prompt_text"]
TARGETS = ["content", "wording"]
FEATURES = [col for col in train.columns if col not in NO_FEATURES + TARGETS]

# 타겟과 피쳐의 피어슨 상관계
corr = train[TARGETS + FEATURES].corr()
corr.style.background_gradient(cmap='coolwarm')

/tmp/ipykernel_27/3379333018.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = train[TARGETS + FEATURES].corr()


,content,wording,num_unique_words,num_words,num_sentences,isupper,mean_num_words,mean_num_unique_words,num_slash,paragraph_count,upper_count,syntax_count,stopwords_rel,diff_emb_sb,diff_emb_qa,automated_readability_index,coleman_liau_index,smog_index,dale_chall_readability_score,linsear_write_formula,gunning_fog,text_standard_float,spache_readability,rix,lix
content,1.000000,0.751380,0.806767,0.792626,0.650758,0.133701,0.072942,0.123573,-0.019386,-0.020365,-0.056648,0.530829,-0.126687,0.584948,0.270435,0.100422,0.183494,0.596993,0.117586,-0.044882,0.074267,0.078944,0.063659,0.138534,0.118763
wording,0.751380,1.000000,0.544271,0.536343,0.453280,0.151596,-0.049175,-0.003145,-0.043161,-0.035557,-0.043505,0.263644,-0.056033,0.339010,0.140404,-0.036973,0.135200,0.523179,0.059624,-0.164171,-0.048049,0.014315,-0.060854,0.030539,-0.010726
num_unique_words,0.806767,0.544271,1.000000,0.981951,0.773660,0.066029,0.188446,0.244561,0.023555,0.007132,-0.045243,0.749152,-0.108114,0.742368,0.210319,0.186192,0.156567,0.537439,0.116307,0.047701,0.154099,0.099468,0.146613,0.172073,0.184885
num_words,0.792626,0.536343,0.981951,1.000000,0.769209,0.054544,0.206381,0.246060,0.019315,0.003430,-0.044073,0.744100,-0.063057,0.757336,0.203075,0.197969,0.120279,0.509318,0.093056,0.057110,0.166637,0.102179,0.158015,0.171245,0.192223
num_sentences,0.650758,0.453280,0.773660,0.769209,1.000000,0.080648,-0.242674,-0.225644,0.024248,0.012686,-0.010049,0.565643,-0.256289,0.677247,0.153578,-0.058515,0.151310,0.489716,0.006866,-0.217835,-0.109972,-0.047456,-0.106300,-0.017172,-0.044364
isupper,0.133701,0.151596,0.066029,0.054544,0.080648,1.000000,-0.130661,-0.107555,-0.023311,-0.009534,0.027713,0.029402,-0.137051,-0.007662,0.090631,-0.141291,-0.006094,0.147117,-0.045785,-0.182541,-0.140825,-0.086375,-0.145968,-0.092650,-0.127770
mean_num_words,0.072942,-0.049175,0.188446,0.206381,-0.242674,-0.130661,1.000000,0.970844,-0.010526,-0.015144,-0.050575,0.170815,0.342177,0.039570,0.037050,0.498151,-0.053386,-0.132648,0.152228,0.587130,0.521360,0.279100,0.508520,0.336280,0.452195
mean_num_unique_words,0.123573,-0.003145,0.244561,0.246060,-0.225644,-0.107555,0.970844,1.000000,-0.009764,-0.014557,-0.051754,0.208938,0.305855,0.049776,0.063561,0.465758,-0.031098,-0.080540,0.149225,0.556384,0.492322,0.271701,0.475676,0.327643,0.424584
num_slash,-0.019386,-0.043161,0.023555,0.019315,0.024248,-0.023311,-0.010526,-0.009764,1.000000,0.810290,-0.001781,0.028108,-0.034564,0.023776,-0.031628,0.000483,0.058423,0.025060,0.019633,-0.013086,-0.007156,0.004467,-0.004362,0.005322,0.003672
paragraph_count,-0.020365,-0.035557,0.007132,0.003430,0.012686,-0.009534,-0.015144,-0.014557,0.810290,1.000000,-0.007934,0.009445,-0.027659,0.013583,-0.028556,-0.005587,0.048951,0.025307,0.014896,-0.021208,-0.012340,-0.004609,-0.009474,-0.001006,-0.000742


Learning catboost and check metric:

특정 그룹 집합에 대해 훈련된 모델이 다른 그룹에 잘 일반화되는지 알고 싶다면 어떻게 검증할 수 있을까? 

이를 측정하려면 validation fold의 모든 샘플이 짝을 이루는 train fold에서 오지 않았는지 확인해야 한다.

GroupKFold는 train, test data에서 동일한 그룹이 표시되지 않도록 하는 KFold의 변형 방식

Regression 기준 기존 부스팅 방법
1. 실제 값들의 평균과 실제 값의 차이인 잔차(Residual)를 구한다.
2. 데이터로 이 잔차들을 학습하는 모델을 만든다.
3. 만든 모델로 예측하여, 예측 값에 Learning_rate 를 곱해 실제 예측 값(평균 + 잔차예측 값*lr) 을 업데이트 한다.
4. 1~3 반복

CatBoostRegressor

기존의 부스팅 모델이 일괄적으로 모든 훈련 데이터를 대상으로 잔차(실제 값들의 평균과 실제 값의 차이)계산을 했다면, Catboost 는 일부만 가지고 잔차계산을 한 뒤, 이걸로 모델을 만들고, 그 뒤에 데이터의 잔차는 이 모델로 예측한 값을 사용

기존의 부스팅 기법은 모든 데이터(x1 ~ x10) 까지의 잔차를 일괄 계산한다.

반면, Catboost 의 과정은 다음과 같다.

1.  먼저 x1 의 잔차만 계산하고, 이를 기반으로 모델을 만든다. 그리고 x2 의 잔차를 이 모델로 예측한다.
2.  x1, x2 의 잔차를 가지고 모델을 만든다. 이를 기반으로 x3, x4 의 잔차를 모델로 예측한다.
3.  x1, x2, x3, x4 를 가지고 모델을 만든다. 이를 기반으로 x5, x6, z7, x8 의 잔차를 모델로 예측한다.
4.  ... 반복

In [25]:
from sklearn.model_selection import GroupKFold # 교차검증
from catboost import CatBoostRegressor


gfk = GroupKFold(n_splits=4)
train_oof = np.zeros((len(train), 2))
test_pred = np.zeros((len(test), 2))
X, y = train[FEATURES], train[TARGETS]
for train_index, val_index in gfk.split(train, groups=train["prompt_id"]):
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_val, y_val = X.iloc[val_index], y.iloc[val_index]
    
    model = CatBoostRegressor(random_state=42, max_depth=4, 
                              objective="MultiRMSE", text_features=["text"])
    model.fit(X_train, y_train, eval_set=(X_val, y_val), metric_period=100)
    
    train_oof[val_index] = model.predict(X_val)
    test_pred +=  model.predict(test[FEATURES])/4

0:	learn: 1.4627694	test: 1.4251682	best: 1.4251682 (0)	total: 210ms	remaining: 3m 29s
100:	learn: 0.8793132	test: 0.8890167	best: 0.8890167 (100)	total: 13.8s	remaining: 2m 2s
200:	learn: 0.8165654	test: 0.8517480	best: 0.8517480 (200)	total: 26s	remaining: 1m 43s
300:	learn: 0.7782378	test: 0.8285230	best: 0.8285230 (300)	total: 38.7s	remaining: 1m 29s
400:	learn: 0.7487944	test: 0.8128298	best: 0.8128298 (400)	total: 50.8s	remaining: 1m 15s
500:	learn: 0.7257715	test: 0.8027143	best: 0.8027143 (500)	total: 1m 2s	remaining: 1m 2s
600:	learn: 0.7075770	test: 0.7960856	best: 0.7960856 (600)	total: 1m 15s	remaining: 50.2s
700:	learn: 0.6923834	test: 0.7919925	best: 0.7919925 (700)	total: 1m 27s	remaining: 37.4s
800:	learn: 0.6789387	test: 0.7908040	best: 0.7908040 (800)	total: 1m 40s	remaining: 25s
900:	learn: 0.6672846	test: 0.7891293	best: 0.7891293 (900)	total: 1m 52s	remaining: 12.4s
999:	learn: 0.6570172	test: 0.7880420	best: 0.7880420 (999)	total: 2m 4s	remaining: 0us

bestTest = 

Checking competition metric:

In [26]:
compute_mcrmse((train_oof, train[TARGETS]))

{'content_rmse': 0.5441777875082165,
 'wording_rmse': 0.7985225258047995,
 'mcrmse': 0.6713501566565081}

Not bad!We need submit it!

In [27]:
sample_submission["content"] = test_pred[:, 0]
sample_submission["wording"] = test_pred[:, 1]

sample_submission.to_csv("submission.csv", index=False)

The main idea of this notebook, inspire comunity to not only train transformers, but to find new interesting solutions!